In [7]:
import os
import requests

In [8]:
import geopandas as gpd
import pandas as pd

## Open State data

In [9]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
state_gdf = gpd.read_file(file_path)

## Open Politics Contributions Data

In [12]:
def format_df(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["Amount"] = df["Amount"].str.replace("$", "").astype(int)
    df = df.groupby("State")["Amount"].sum().reset_index(name="SUM")
    return df

In [13]:
dfs = [format_df(f"data/{csv}") for csv in os.listdir("data") if csv.endswith("csv")]

In [14]:
merged = pd.concat(dfs)
money_df = merged.groupby("State", as_index=False)["SUM"].sum()
money_df = money_df.rename(columns={"State": "NAME"})

## Merge Data

In [15]:
congress_gdf = state_gdf.merge(money_df, on="NAME", how="left")

In [16]:
congress_gdf = congress_gdf.fillna(0)

In [17]:
congress_gdf = congress_gdf.to_crs(9311)
congress_gdf.to_file("data/Dairy_Lobby_Contributions_Per_State.gpkg")

In [18]:
congress_gdf["SUM"].sum()

np.float64(2519445.0)